In [11]:
# -*- coding: utf-8 -*-
# VERSÃO GRATUITA — Graham Clássico + Bazin com seletor de DPS (com Etiqueta para Graham e Bazin)
# Uso:
#   pip install yfinance pandas numpy
#   python gratuito_basico.py

import numpy as np
import pandas as pd
import yfinance as yf

# ==============================
# , "BBAS3.SA", "TAEE11.SA", "CXSE3.SA", "VALE3.SA", "SAPR4.SA", "CMIG4.SA", "SANB4.SA"
TICKERS = ["KLBN4.SA", "PETR4.SA", "ITSA4.SA", "ISAE4.SA"]
BASE_DPS = "MEDIA_3A"   # opções: "12M", "LTM", "MEDIA_3A"
MOS = 0.20              # margem de segurança para as Etiquetas (20%)
# ==============================

def ensure_naive_datetime_index(s: pd.Series) -> pd.Series:
    if s is None or len(s) == 0:
        return s
    idx = pd.DatetimeIndex(s.index)
    if idx.tz is not None:
        idx = idx.tz_convert("UTC").tz_localize(None)
    else:
        idx = idx.tz_localize(None) if getattr(idx, "tz", None) else idx
    return pd.Series(s.values, index=idx, name=getattr(s, "name", None))

def dps_12m(divs: pd.Series) -> float:
    if divs is None or len(divs) == 0:
        return np.nan
    divs = ensure_naive_datetime_index(divs)
    cutoff = pd.Timestamp.utcnow().tz_localize(None) - pd.DateOffset(days=365)
    return float(divs.loc[divs.index >= cutoff].sum())

def dps_ltm(divs: pd.Series) -> float:
    if divs is None or len(divs) == 0:
        return np.nan
    divs = ensure_naive_datetime_index(divs).to_frame("v")
    divs["quarter"] = divs.index.to_period("Q")
    ult4 = divs["quarter"].unique()[-4:]  # últimos 4 trimestres
    return float(divs.loc[divs["quarter"].isin(ult4), "v"].sum())

def dps_media_3a(divs: pd.Series) -> float:
    if divs is None or len(divs) == 0:
        return np.nan
    divs = ensure_naive_datetime_index(divs)
    cutoff = pd.Timestamp.utcnow().tz_localize(None) - pd.DateOffset(years=3)
    sub = divs.loc[divs.index >= cutoff]
    if len(sub) == 0:
        return np.nan
    return float(sub.sum()) / 3.0

def graham_classico(eps, bvps):
    if eps and bvps and eps > 0 and bvps > 0:
        return (22.5 * eps * bvps) ** 0.5
    return np.nan

def preco_teto_bazin(dps):
    if dps and dps > 0:
        return dps / 0.06
    return np.nan

def upside(preco_atual, preco_justo):
    if preco_atual and preco_justo and preco_justo > 0:
        return (preco_justo / preco_atual - 1.0) * 100.0
    return np.nan

def etiqueta(preco_atual, preco_justo, mos=0.20):
    if (preco_atual is None) or (preco_justo is None) or np.isnan(preco_justo) or preco_justo <= 0:
        return "Sem dados"
    rel = float(preco_atual) / float(preco_justo)
    if rel <= (1 - mos): return "Barata"
    if rel <= (1 + mos): return "Justa"
    return "Cara"

def analyze(tk):
    t = yf.Ticker(tk)
    info = t.info or {}
    price = info.get("currentPrice")
    eps = info.get("trailingEps")
    bvps = info.get("bookValue")
    try:
        divs = t.dividends
    except Exception:
        divs = pd.Series(dtype=float)

    # Escolher base de cálculo do DPS
    if BASE_DPS == "12M":
        dps = dps_12m(divs)
    elif BASE_DPS == "LTM":
        dps = dps_ltm(divs)
    elif BASE_DPS == "MEDIA_3A":
        dps = dps_media_3a(divs)
    else:
        dps = dps_12m(divs)

    graham = graham_classico(eps, bvps)
    preco_teto = preco_teto_bazin(dps)

    return {
        "ticker": tk,
        "Cotação R$": price,
        "LPA": eps,
        "VPA": bvps,
        "DY(3 anos)": dps,
        "Preço Justo(Graham)": graham,
        "Upside %(Graham)": upside(price, graham),
        "Situação(Graham)": etiqueta(price, graham, MOS),   # <<< NOVA COLUNA
        "Preço-teto(Bazin)": preco_teto,
        "Upside %(Bazin)": upside(price, preco_teto),
        "Situação(Bazin)": etiqueta(price, preco_teto, MOS),
    }

if __name__ == "__main__":
    rows, fails = [], []
    for tk in TICKERS:
        try:
            rows.append(analyze(tk))
        except Exception as e:
            fails.append((tk, str(e)))

    df = pd.DataFrame(rows)
    print(f"\n=== RESULTADOS GRATUITOS (BASE = {BASE_DPS}) ===")
    print(df.to_string(index=False))

    df.to_csv("gratuito_resultados.csv", index=False)
    print("\nArquivo salvo: gratuito_resultados.csv")
    if fails:
        print("Falhas:", fails)

# Disclaimer (para imprimir no final):
print("\n Aviso: Os resultados têm caráter exclusivamente informativo e educacional. "
      "\n Não constituem recomendação de compra ou venda de ativos. "
      "\n Faça suas próprias análises ou consulte um profissional qualificado.")



=== RESULTADOS GRATUITOS (BASE = MEDIA_3A) ===
  ticker  Cotação R$  LPA    VPA  DY(3 anos)  Preço Justo(Graham)  Upside %(Graham) Situação(Graham)  Preço-teto(Bazin)  Upside %(Bazin) Situação(Bazin)
KLBN4.SA        3.64 0.34  1.678    0.235211             3.582834         -1.570492            Justa           3.920183         7.697344           Justa
PETR4.SA       31.00 5.71  5.676    6.451114            27.004150        -12.889840            Justa         107.518561       246.834068          Barata
ITSA4.SA       11.22 1.39  8.262    0.780167            16.074640         43.267736           Barata          13.002778        15.889285           Justa
ISAE4.SA       24.53 4.93 31.268    1.757732            58.893148        140.086213           Barata          29.295533        19.427368           Justa

Arquivo salvo: gratuito_resultados.csv

 Aviso: Os resultados têm caráter exclusivamente informativo e educacional. 
 Não constituem recomendação de compra ou venda de ativos. 
 Faça sua